# Main Code

In [3]:
# Installing packages
#!pip install pandas openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [1]:
# Loading packages
import pandas as pd
import numpy as np

### Loading data files

In [52]:
# Loading data
data = pd.read_excel("Data/Data_final.xlsx") 

# Display the first 5 rows
print(data.head())

metadata = pd.read_excel("Data/Metadata_final.xlsx") 

# Display the first 5 rows  
print(metadata.head())

         Sample_Name  SeaDistance_m Location Duplicate    Datetime   Latitude  \
0  SLWT_2024_01_19_A            NaN    shore         A  2024_01_19  16.625000   
1  SLWT_2024_01_19_B            NaN    shore         B  2024_01_19  16.625000   
2  SLWT_2024_01_20_A            NaN    shore         A  2024_01_20  16.401667   
3  SLWT_2024_01_20_B            NaN    shore         B  2024_01_20  16.401667   
4  SLWT_2024_01_21_A            NaN    shore         A  2024_01_21  16.316667   

   Longitude  Type  HP_12CH4_dry_mean  HR_12CH4_dry_mean  ...  AirT_degC  \
0 -25.325000   NaN           1.854859           1.838359  ...       24.5   
1 -25.325000   NaN           1.729128           1.715480  ...       24.5   
2 -27.518333   NaN           1.827733           1.818951  ...       23.5   
3 -27.518333   NaN           1.892697           1.893364  ...       23.5   
4 -30.581667   NaN           1.902200           1.882740  ...       25.0   

   AirP_hPa  CH4atm_ppm  CO2atm_ppm  d13C_CH4atm_permil 

### Loading Variables

In [53]:
# Some usefull variables

R_JKmol = 8.314 # [J/K/mol], ideal gas law
V_w_L = 0.105 #[L], volume of the water sample in the syringe
V_hs_L = 0.35 #[L], volume of the headspace in the syringe
V_tot_L = 0.140 #[L], volume total of the syringe
P_atm_Pa = 101325 #[Pa], atmospheric pressure

### Calculation of ocean-air fluxes for CO2 and methane

Partial pressure - Pi [Pa]

In [54]:
# CO2
Pi_CO2_Pa = data["CO2atm_ppm"] * 10**(-6) * P_atm_Pa
#print(Pi_CO2_Pa)

#CH4
Pi_CH4_Pa = data["CH4atm_ppm"] * 10**(-6) * P_atm_Pa
#print(Pi_CH4_Pa)

Gas concentration in the water - n [mol]

In [55]:
#convert Temp at equilibrium (assumed to be the Temp after shaking) in Kelvin
Te_K = data["Te_degC"] + 273.15 
#print(Te_K)

In [56]:
# Ideal Gas Law (Magen et al., 2014) 
#converting V_w_L in m3
V_w_m3 = V_w_L / 1000

n_CO2_mol = Pi_CO2_Pa * V_w_m3 / (R_JKmol * Te_K)
#print(n_CO2_mol)

n_CH4_mol = Pi_CH4_Pa * V_w_m3 / (R_JKmol * Te_K)
#print(n_CH4_mol)

Corrected Henry's coefficients [mol/m3/Pa]

In [57]:
# CO2 (Weiss 1974)
A = [-58.0931, 90.5069, 22.2940]
B = [0.027766, -0.025888, 0.0050578]
hcpsalt_CO2_molm3Pa = (
    np.exp(
    A[0]
    + A[1] * 100 / Te_K
    + A[2] * np.log(Te_K / 100)
    + data["sal_psu"] * (B[0] + B[1] * Te_K / 100 + B[2] * (Te_K / 100) ** 2)
    ) * 1000/ 101325 
    )
#print(hcpsalt_CO2_molm3Pa)

In [58]:
# CH4 (Wiesenburg and Guinasso 1979)
A = [-417.5053, 599.8626, 380.3636, -62.0764]
B = [-0.064236, 0.03498, -0.0052732]
c_molm3 = (
    np.exp(
    + A[0]
    + A[1] * 100 / Te_K
    + A[2] * np.log(Te_K / 100)
    + A[3] * (Te_K / 100)
    + data["sal_psu"] * (B[0] + B[1] * Te_K / 100 + B[2] * (Te_K / 100) ** 2)
    ) * 1000 / 1e9
    )

hcpsalt_CH4_molm3Pa = c_molm3 / 101325

#print(hcpsalt__CH4_molm3Pa)

Dissolved water concentration [mol/m3] or solubility of the gases

In [59]:
# Henry’s Law (Sander, 2023)
# CO2
C_eq_CO2_molm3 = hcpsalt_CO2_molm3Pa * Pi_CO2_Pa
#print(C_eq_CO2_molm3)

# CH4
C_eq_CH4_molm3 = hcpsalt_CH4_molm3Pa * Pi_CH4_Pa
#print(C_eq_CH4_molm3)

Gas transfer velocity K600 [m/s]

In [60]:
# Function to extract min and max wind speed
kts_to_ms = 1852 / 3600

def WS_min_max(wind_speed):
    if isinstance(wind_speed, str) and '-' in wind_speed:  # Check if it's a range
        min_val, max_val = map(float, wind_speed.split('-'))  # Split and convert to float
    else:
        min_val = max_val = float(wind_speed)  # If it's a single number
    return pd.Series([min_val, max_val])

# Apply function to create new columns
metadata[['Min_Wind_Speed [kts]', 'Max_Wind_Speed [kts]']] = metadata['TWS [kts]'].apply(WS_min_max)

# Convert knots to meters per second
metadata['Min_Wind_Speed [m/s]'] = metadata['Min_Wind_Speed [kts]'] * kts_to_ms
metadata['Max_Wind_Speed [m/s]'] = metadata['Max_Wind_Speed [kts]'] * kts_to_ms

# Display the first rows
print(metadata.head())

         Sample_Name   Latitude  Longitude TWS [kts]  SOG [kts]  P_atm [hPa]  \
0  SLWT_2024_01_19_A  16.625000 -25.325000         7        4.5       1016.0   
1  SLWT_2024_01_19_B  16.625000 -25.325000         7        4.5       1016.0   
2  SLWT_2024_01_20_A  16.401667 -27.518333        16        7.6       1014.7   
3  SLWT_2024_01_20_B  16.401667 -27.518333        16        7.6       1014.7   
4  SLWT_2024_01_21_A  16.316667 -30.581667        19        7.5       1014.9   

   T_air [°C]   pH  Tw_b [°C]  Tw_a [°C] Mer [Douglas]  \
0        24.5  7.5       24.0       23.5             3   
1        24.5  7.5       23.5       23.5             3   
2        23.5  7.5       24.8       25.4           3-4   
3        23.5  7.5       24.5       25.2           3-4   
4        25.0  7.5       24.7       25.3             3   

                           Remarks  Min_Wind_Speed [kts]  \
0  Air pressure from NOAA archives                   7.0   
1  Air pressure from NOAA archives                

In [61]:
# Calculation of the gas transfer velocity K600 (MacIntyre et al., 2010) 
K600_min_ms = 2.04 * 10 * metadata["Min_Wind_Speed [m/s]"] + 2.0
#print(K600_min_ms)

K600_max_ms = 2.04 * 10 * metadata["Max_Wind_Speed [m/s]"] + 2.0
#print(K600_max_ms)

Schmidt number - Sc [-]

In [62]:
# (Wanninkhof, 2014)
#CO2
Sc_CO2 =  2073.1 - 125.62 * metadata["Tw_b [°C]"] + 3.6276 * metadata["Tw_b [°C]"]**2 - 0.043219 * metadata["Tw_b [°C]"]**3
#print(Sc_CO2)

#CH4
Sc_CH4 = 2039.2 - 120.31 * metadata["Tw_b [°C]"] + 3.4209 * metadata["Tw_b [°C]"]**2 - 0.040437 * metadata["Tw_b [°C]"]**3
#print(Sc_CH4)

Exchange coefficient - Ki [m/d]

In [63]:
# Define n from the gas exchange equation (n = 0.5 for wind speeds > 3.7 [m s-1] or, n = 0.66 for wind speeds < 3.7 [m s-1]) :
def WS_n (wind_speed) :
    if wind_speed < 3.7 :
        n = 0.66
    else:
        n = 0.5
    return n

metadata['Min_n'] = metadata['Min_Wind_Speed [m/s]'].apply(WS_n)
metadata['Max_n'] = metadata['Max_Wind_Speed [m/s]'].apply(WS_n)

print(metadata.head())

         Sample_Name   Latitude  Longitude TWS [kts]  SOG [kts]  P_atm [hPa]  \
0  SLWT_2024_01_19_A  16.625000 -25.325000         7        4.5       1016.0   
1  SLWT_2024_01_19_B  16.625000 -25.325000         7        4.5       1016.0   
2  SLWT_2024_01_20_A  16.401667 -27.518333        16        7.6       1014.7   
3  SLWT_2024_01_20_B  16.401667 -27.518333        16        7.6       1014.7   
4  SLWT_2024_01_21_A  16.316667 -30.581667        19        7.5       1014.9   

   T_air [°C]   pH  Tw_b [°C]  Tw_a [°C] Mer [Douglas]  \
0        24.5  7.5       24.0       23.5             3   
1        24.5  7.5       23.5       23.5             3   
2        23.5  7.5       24.8       25.4           3-4   
3        23.5  7.5       24.5       25.2           3-4   
4        25.0  7.5       24.7       25.3             3   

                           Remarks  Min_Wind_Speed [kts]  \
0  Air pressure from NOAA archives                   7.0   
1  Air pressure from NOAA archives                

In [64]:
# Find indices where Min and Max n are different
indices = metadata[metadata["Min_n"] != metadata["Max_n"]].index
print(indices)

Index([30, 31], dtype='int64')


In [68]:
# Gas exchange for CO2 (McGinnis et al., 2015) 
Ki_min_CO2_md = K600_min_ms * (600 / Sc_CO2)**(-metadata["Min_n"])
#print(Ki_min_CO2_md)

Ki_max_CO2_md = K600_max_ms * (600 / Sc_CO2)**(-metadata["Max_n"])
#print(Ki_max_CO2_md)

In [66]:
# Gas exchange for CH4 (McGinnis et al., 2015) 
Ki_min_CH4_md = K600_min_ms * (600 / Sc_CH4)**(-metadata["Min_n"])
#print(Ki_min_CH4_md)

Ki_max_CH4_md = K600_max_ms * (600 / Sc_CH4)**(-metadata["Max_n"])
#print(Ki_max_CH4_md)

Ocean-air fluxes - F [mmol/m2/d] 

In [69]:
#Fick’s first law for CO2 (Fick, 1855) 
F_min_CO2_mmolm2d = Ki_min_CO2_md * (data["co2_mmolm3"] - C_eq_CO2 * 10**3)
print(F_min_CO2_mmolm2d)

F_max_CO2_mmolm2d = Ki_max_CO2_md * (data["co2_mmolm3"] - C_eq_CO2 * 10**3)
print(F_max_CO2_mmolm2d)

#mean values
mean_F_min_CO2_mmolm2d = np.mean(F_min_CO2_mmolm2d)
mean_F_max_CO2_mmolm2d = np.mean(F_max_CO2_mmolm2d)
print("Mean min CO2 flux:", mean_F_min_CO2_mmolm2d)
print("Mean max CO2 flux:", mean_F_max_CO2_mmolm2d)

0      302.317406
1      552.171207
2      573.398330
3      595.466927
4       52.638264
5      143.268108
6      245.246988
7      204.061557
8      154.897059
9       64.520838
10      38.572854
11      88.282783
12     297.218581
13     149.168048
14     198.204694
15     154.968812
16     208.805176
17       9.178345
18      71.032296
19      22.043624
20     202.437489
21     110.794598
22       8.010852
23      21.762481
24      51.717529
25     341.587207
26     171.902027
27     206.816201
28    -731.752472
29    -784.267008
30    -179.819879
31    -272.093715
32    -396.290993
33    -271.685880
34   -1877.945809
35   -2141.136289
36   -1149.005492
37    -791.113654
38    -116.076624
39      99.661744
40     -72.861215
41    -551.440358
42    -461.160663
43    -230.564506
44    -100.556752
45    -881.631149
46       4.075339
47    -108.411041
48      10.065406
49       0.775067
50      -0.665449
dtype: float64
0      302.317406
1      552.171207
2      573.398330
3      595.46

In [71]:
#Fick’s first law for CH4 (Fick, 1855)
F_min_CH4_umolm2d = Ki_min_CH4_md * (data["ch4_mmolm3"] - C_eq_CH4 * 10**(3)) * 10**3
print(F_min_CH4_umolm2d)

F_max_CH4_umolm2d = Ki_max_CH4_md * (data["ch4_mmolm3"] - C_eq_CH4 * 10**(3)) * 10**3
print(F_max_CH4_umolm2d)

#mean values
mean_F_min_CH4_umolm2d = np.mean(F_min_CH4_umolm2d)
mean_F_max_CH4_umolm2d = np.mean(F_max_CH4_umolm2d)
print("Mean min CH4 flux:", mean_F_min_CH4_umolm2d)
print("Mean max CH4 flux:", mean_F_max_CH4_umolm2d)

0      524.259220
1      391.885438
2      406.759620
3      572.001057
4      222.543107
5      224.563880
6       38.567153
7       85.245167
8       55.322180
9      -85.376458
10       3.337389
11   -1205.363436
12   -1294.861429
13     107.079618
14     172.664243
15     103.545551
16     124.155236
17     125.883856
18    1413.716431
19      87.797959
20     104.317302
21     -15.974379
22     229.924628
23     160.723811
24     213.314515
25      36.432490
26      84.372825
27      93.783730
28     148.991191
29     166.840441
30    1481.478847
31     210.342070
32    -182.933175
33      90.516880
34      27.424525
35     357.264058
36      92.492718
37     333.893937
38     153.756487
39     192.990665
40      37.060841
41      25.884033
42     140.000415
43      76.442760
44     141.999643
45     336.700110
46     192.680267
47     236.129975
48      49.248103
49      65.377620
50      57.838307
dtype: float64
0      524.259220
1      391.885438
2      406.759620
3      572.00